# Setting Soil Properties for Shetran

This notebook was created to demostrate the process of creating the soil properties file use in SHETRAN based in the catchment extent

---

#### Author: 
                LF Velasquez & I Rohrmueller - Newcastle University

#### Date:
                Nov 2022

#### Version:
                1.0

#### Notes:
                - To get jupyter env version type `!jupyter --version` in a python cell
            
#### Jupyter version:

#### Python version:

---

# Notebook set-up

## 1. Setting Python Modules

In [1]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import rasterstats
import rioxarray
import numpy as np

# XML libraries
from dicttoxml import dicttoxml
import xml.etree.ElementTree as ET
# from dict2xml import dict2xml
from xml.dom.minidom import parseString

# ODC modules
import datacube

# Custom modules
import utils

# Modules for config.ini
from configparser import ConfigParser
config = ConfigParser()

## 2. Global variables

In [2]:
# Setting the path to the work environment
dir_abs = Path().resolve().parent.parent

# Make sure temp folder directory is empty
utils.file_remove(Path(dir_abs / 'shetran_data/temp_data/'), 'all')

# Read config file values
config.read('config.ini')

# Setting CRS
crs_global = config.getint('crs_setting', 'GLB')
crs_local = config.getint('crs_setting', 'COL')

# No data value
ND = config.getint('res_setting', 'NO_DATA')

# Open Data Cube Product
lst_dc_data = config.get('dc_product', 'lst_soil_products').split(',')

/home/geofelpave/Documents/1_PhD_SharePoint/LFPV - PhD - Documents/00_PhD_main/006_GitHub/0065_odc_hydro/shetran_data/temp_data/2m_temperature_min_20220106.tif DELETED!
/home/geofelpave/Documents/1_PhD_SharePoint/LFPV - PhD - Documents/00_PhD_main/006_GitHub/0065_odc_hydro/shetran_data/temp_data/2m_temperature_min_20220122.tif DELETED!
/home/geofelpave/Documents/1_PhD_SharePoint/LFPV - PhD - Documents/00_PhD_main/006_GitHub/0065_odc_hydro/shetran_data/temp_data/2m_temperature_min_20220524.tif DELETED!
/home/geofelpave/Documents/1_PhD_SharePoint/LFPV - PhD - Documents/00_PhD_main/006_GitHub/0065_odc_hydro/shetran_data/temp_data/2m_temperature_min_20220107.tif DELETED!
/home/geofelpave/Documents/1_PhD_SharePoint/LFPV - PhD - Documents/00_PhD_main/006_GitHub/0065_odc_hydro/shetran_data/temp_data/2m_temperature_min_20220518.tif DELETED!
/home/geofelpave/Documents/1_PhD_SharePoint/LFPV - PhD - Documents/00_PhD_main/006_GitHub/0065_odc_hydro/shetran_data/temp_data/2m_temperature_min_20220522

## 3. Start of Process

### 3.1. Reading shapefile

In [3]:
# Read shp file to geopandas dataframe
grid_path = Path(dir_abs / 'shetran_data/active_data/final_mask_wgs84.shp')
grid = gpd.read_file(grid_path)

xmin, ymin, xmax, ymax = grid.total_bounds
print(f'xmin = {xmin}, ymin = {ymin}, xmax = {xmax}, ymax = {ymax}')

xmin = -76.70463049691715, ymin = 3.211597459533565, xmax = -76.020367156035, ymax = 3.592734529578263


### 3.2. Reading Soil Property data from ODC

In [145]:
lst_products = ['hihydrosoil_ALFA_0_5cm', 'hihydrosoil_ALFA_100_200cm', 
                'hihydrosoil_ALFA_15_30cm', 'hihydrosoil_ALFA_30_60cm', 
                'hihydrosoil_ALFA_5_15cm', 'hihydrosoil_ALFA_60_100cm', 
                'hihydrosoil_Ksat_0_5cm', 'hihydrosoil_Ksat_100_200cm', 
                'hihydrosoil_Ksat_15_30cm', 'hihydrosoil_Ksat_30_60cm', 
                'hihydrosoil_Ksat_5_15cm', 'hihydrosoil_Ksat_60_100cm', 
                'hihydrosoil_N_0_5cm', 'hihydrosoil_N_100_200cm', 
                'hihydrosoil_N_15_30cm', 'hihydrosoil_N_30_60cm', 
                'hihydrosoil_N_5_15cm', 'hihydrosoil_N_60_100cm', 
                'hihydrosoil_WCres_0_5cm', 'hihydrosoil_WCres_100_200cm', 
                'hihydrosoil_WCres_15_30cm', 'hihydrosoil_WCres_30_60cm', 
                'hihydrosoil_WCres_5_15cm', 'hihydrosoil_WCres_60_100cm', 
                'hihydrosoil_WCsat_0_5cm', 'hihydrosoil_WCsat_100_200cm', 
                'hihydrosoil_WCsat_15_30cm', 'hihydrosoil_WCsat_30_60cm', 
                'hihydrosoil_WCsat_5_15cm', 'hihydrosoil_WCsat_60_100cm'
                ]



### 3.3. Create functions needed

In [4]:
def hihydrosoils(dc_connection, product_name):
    # Get data
    # Load data from the datacube
    print(product_name)
    buffer = 0.125
    ds = dc_connection.load(product=product_name,
                lat=(ymin - buffer, ymax + buffer),
                lon=(xmin - buffer, xmax + buffer),
                )
    return ds

def soil_prop_stats(file_name, data_array, variable_name, path_grid, depth):
    
    # set dataframe
    temp_stats_df = pd.DataFrame()
    
    #create temp .tif file
    _tif = Path(dir_abs / f'shetran_data/temp_data/{file_name}.tif')
    data_array[variable_name].rio.to_raster(_tif)
    
    # Find the mean elevation for each grid in the catchment
    # This produces a dict following the same order as the grid geodataframe
    # soilt_stats = rasterstats.zonal_stats(str(path_grid), str(_tif), categorical=True, nodata=-999)
    soil_stats =  rasterstats.zonal_stats(str(path_grid), str(_tif), stats="mean", all_touched=True, nodata=-9999)

    # Create a list containing the dictionary key (land cover type) with the Max value for each grid in the catchment
    prop_mean = [x['mean'] for x in soil_stats]

    # Add the largest lc to grid geodataframe
    grid['property_value'] = prop_mean
    
    if depth != '200_2000cm':
        
        # change variables to expected number by multiplying by 0.0001
        grid['property_value'] = grid['property_value'] * 0.0001
    else:
        grid['property_value'] = (10**(grid['property_value'] / 100))* 100000000000

    # # Replacing 0 with -9999
    # grid.loc[grid[variable_name] == 0, variable_name] = -9999
    
    # delte temp .tif file
    utils.file_remove(Path(dir_abs / 'shetran_data/temp_data/'), 'tif')
    # utils.file_remove(_tif)

    return grid

def soil_prop_dataframe(soil_grid_df, crs_num, variable_name, depth):
    # Get the centroid for each grid
    temp_df = pd.DataFrame()
    temp_df = soil_grid_df.to_crs(crs_num)
    temp_df['centroid'] = temp_df['geometry'].centroid

    # Create lat (Y) and lon (X) columns
    temp_df['lat'] = temp_df['centroid'].y.astype(int)
    temp_df['lon'] = temp_df['centroid'].x.astype(int)

    # Change geopandas to pandas ready to create csv file for the mean and min elevation
    # df_grid_lc = pd.DataFrame(temp_df[['lat', 'lon', 'property_value']].copy())
    df_grid_lc = pd.DataFrame(temp_df[['lat', 'lon', 'SHETRAN_ID', 'property_value']].copy())
    
    # add the soil depth for reference
    df_grid_lc['soil_depth'] = depth
    
    # add the soil property name for reference
    df_grid_lc['property_name'] = variable_name
    
    return df_grid_lc
    


### 3.4 Run process
This creates a list of dataframes, each dataframe represents a soil variable

#### 3.4.1. Create column using hihydrosoil

In [5]:
df_soil = pd.DataFrame()

# Set ODC application
dc = datacube.Datacube(app="hihydrosoil")

# list to stor dataframe
lst_df = []

for count, value in enumerate(lst_dc_data):
    # Set variables
    var_name = value.split("_")[1]
    soil_depth = '_'.join(value.split("_")[2:])
    
    # Get data from ODC
    hihydrosoil_var = hihydrosoils(dc, value)

    # Calculate soil stats per grid
    df_properties = soil_prop_stats(value, hihydrosoil_var, var_name, grid_path, soil_depth)

    # Prepare soil property dataframe
    df_soil = soil_prop_dataframe (df_properties, crs_local, var_name, soil_depth)
    
    # store final output in list
    lst_df.append(df_soil)

hihydrosoil_ALFA_0_5cm
hihydrosoil_ALFA_0_5cm DELETED!
hihydrosoil_ALFA_100_200cm
hihydrosoil_ALFA_100_200cm DELETED!
hihydrosoil_ALFA_15_30cm
hihydrosoil_ALFA_15_30cm DELETED!
hihydrosoil_ALFA_30_60cm
hihydrosoil_ALFA_30_60cm DELETED!
hihydrosoil_ALFA_5_15cm
hihydrosoil_ALFA_5_15cm DELETED!
hihydrosoil_ALFA_60_100cm
hihydrosoil_ALFA_60_100cm DELETED!
hihydrosoil_Ksat_0_5cm
hihydrosoil_Ksat_0_5cm DELETED!
hihydrosoil_Ksat_100_200cm
hihydrosoil_Ksat_100_200cm DELETED!
hihydrosoil_Ksat_15_30cm
hihydrosoil_Ksat_15_30cm DELETED!
hihydrosoil_Ksat_30_60cm
hihydrosoil_Ksat_30_60cm DELETED!
hihydrosoil_Ksat_5_15cm
hihydrosoil_Ksat_5_15cm DELETED!
hihydrosoil_Ksat_60_100cm
hihydrosoil_Ksat_60_100cm DELETED!
hihydrosoil_N_0_5cm
hihydrosoil_N_0_5cm DELETED!
hihydrosoil_N_100_200cm
hihydrosoil_N_100_200cm DELETED!
hihydrosoil_N_15_30cm
hihydrosoil_N_15_30cm DELETED!
hihydrosoil_N_30_60cm
hihydrosoil_N_30_60cm DELETED!
hihydrosoil_N_5_15cm
hihydrosoil_N_5_15cm DELETED!
hihydrosoil_N_60_100cm
hihydr

#### 3.4.2. Create Geology layer for column

In [6]:
df_properties = pd.DataFrame()
df_soil = pd.DataFrame()

# Set variables
value = 'ghlymps_logKferr_200_2000cm'
var_name = value.split("_")[1]
soil_depth = '_'.join(value.split("_")[2:])

# Get data from ODC
glhymps_var = hihydrosoils(dc, value)

# Calculate soil stats per grid
df_properties = soil_prop_stats(value, glhymps_var, var_name, grid_path, soil_depth)

# Prepare Ksat dataframe
df_soil = soil_prop_dataframe (df_properties, crs_local, 'Ksat', soil_depth)

# Add data to the list of dataframes
lst_df.append(df_soil)

# Create dataframe for other properties at the same depth
prop_lst = ['ALFA','N','WCres','WCsat']
prop_values = [0.01,5.0,0.2,0.3]

for count, value in enumerate(prop_lst):
    # copy the Ksat dataframe to create the other
    # this only works because the soil properties are constant across grids
    df_temp = pd.DataFrame() # create clear df for each loop iteration
    df_temp = df_soil.copy()
    
    # Replace values in dataframe
    df_temp['property_value'] = prop_values[count]
    df_temp['property_name'] = value
    
    # add data to the list of dataframes 
    lst_df.append(df_temp)
    
print ('Geology has been created')

ghlymps_logKferr_200_2000cm
ghlymps_logKferr_200_2000cm DELETED!
Geology has been created


#### Combine all dataframes into a single dataframe

In [8]:
# combine all dataframe
df_soil_all = pd.concat(lst_df)
df_soil_all = df_soil_all.reset_index()

# Round property values
df_soil_all['property_value'] = df_soil_all['property_value'].round(2)

df_soil_all


,index,lat,lon,SHETRAN_ID,property_value,soil_depth,property_name
0,0,888266,709018,-9999,0.01,0_5cm,ALFA
1,1,886266,709018,-9999,0.01,0_5cm,ALFA
2,2,884266,709018,-9999,0.01,0_5cm,ALFA
3,3,882266,709018,-9999,0.01,0_5cm,ALFA
4,4,880266,709018,-9999,0.01,0_5cm,ALFA
...,...,...,...,...,...,...,...
27925,793,856266,783018,0,0.30,200_2000cm,WCsat
27926,794,854266,783018,-9999,0.30,200_2000cm,WCsat
27927,795,852266,783018,-9999,0.30,200_2000cm,WCsat
27928,796,850266,783018,-9999,0.30,200_2000cm,WCsat


#### Pivot table to have soil properties as columns

In [9]:
pivoted = df_soil_all.pivot(index=['lat','lon', 'soil_depth', 'SHETRAN_ID'], columns='property_name', values='property_value').reset_index()
pivoted

property_name,lat,lon,soil_depth,SHETRAN_ID,ALFA,Ksat,N,WCres,WCsat
0,848266,709018,0_5cm,-9999,0.01,5.82,1.36,0.04,0.61
1,848266,709018,100_200cm,-9999,0.01,2.12,1.42,0.04,0.57
2,848266,709018,15_30cm,-9999,0.02,6.15,1.38,0.04,0.59
3,848266,709018,200_2000cm,-9999,0.01,0.03,5.00,0.20,0.30
4,848266,709018,30_60cm,-9999,0.01,2.09,1.43,0.04,0.59
...,...,...,...,...,...,...,...,...,...
5581,888266,783018,15_30cm,-9999,0.02,8.48,1.38,0.04,0.57
5582,888266,783018,200_2000cm,-9999,0.01,0.00,5.00,0.20,0.30
5583,888266,783018,30_60cm,-9999,0.02,3.38,1.44,0.04,0.57
5584,888266,783018,5_15cm,-9999,0.02,8.41,1.37,0.04,0.58


#### Order pivot table
This is use to keep a nice sequential order in the types of soil

In [10]:
pivoted_order = pivoted.copy()
pivoted_order = pivoted_order.sort_values(by=['lat', 'lon', 'soil_depth'])


## Create the depth column
layer_order = [1, 2, 3, 4, 5, 6, 7]
depth_conditions = [
    pivoted_order['soil_depth'] == '0_5cm', 
    pivoted_order['soil_depth'] == '5_15cm',
    pivoted_order['soil_depth'] == '15_30cm',
    pivoted_order['soil_depth'] == '30_60cm',
    pivoted_order['soil_depth'] == '60_100cm',
    pivoted_order['soil_depth'] == '100_200cm',
    pivoted_order['soil_depth'] == '200_2000cm'
    ]

# Adding the soil depth column
pivoted_order['layer_order'] = np.select(depth_conditions, layer_order)

# Sort by category and soil layer
pivoted_order = pivoted_order.sort_values(by=['lat', 'lon', 'layer_order'])
pivoted_order.reset_index(drop=True, inplace=True)

# Drop unnecessary columns
pivoted_order = pivoted_order.drop('layer_order', axis=1)

pivoted_order

property_name,lat,lon,soil_depth,SHETRAN_ID,ALFA,Ksat,N,WCres,WCsat
0,848266,709018,0_5cm,-9999,0.01,5.82,1.36,0.04,0.61
1,848266,709018,5_15cm,-9999,0.02,5.82,1.36,0.04,0.60
2,848266,709018,15_30cm,-9999,0.02,6.15,1.38,0.04,0.59
3,848266,709018,30_60cm,-9999,0.01,2.09,1.43,0.04,0.59
4,848266,709018,60_100cm,-9999,0.01,2.04,1.42,0.04,0.57
...,...,...,...,...,...,...,...,...,...
5581,888266,783018,15_30cm,-9999,0.02,8.48,1.38,0.04,0.57
5582,888266,783018,30_60cm,-9999,0.02,3.38,1.44,0.04,0.57
5583,888266,783018,60_100cm,-9999,0.02,3.62,1.44,0.04,0.56
5584,888266,783018,100_200cm,-9999,0.02,3.82,1.44,0.04,0.55


Check how the column on soil look for a particular grid

In [11]:
pivoted_order[(pivoted_order.lat == 848266) & (pivoted_order.lon == 709018)]

property_name,lat,lon,soil_depth,SHETRAN_ID,ALFA,Ksat,N,WCres,WCsat
0,848266,709018,0_5cm,-9999,0.01,5.82,1.36,0.04,0.61
1,848266,709018,5_15cm,-9999,0.02,5.82,1.36,0.04,0.60
2,848266,709018,15_30cm,-9999,0.02,6.15,1.38,0.04,0.59
3,848266,709018,30_60cm,-9999,0.01,2.09,1.43,0.04,0.59
4,848266,709018,60_100cm,-9999,0.01,2.04,1.42,0.04,0.57
5,848266,709018,100_200cm,-9999,0.01,2.12,1.42,0.04,0.57
6,848266,709018,200_2000cm,-9999,0.01,0.03,5.00,0.20,0.30


#### Set dataframe ready to define soil categories

In [12]:
lst_cols =['lat', 'lon', 'ALFA', 'Ksat', 'N', 'WCres', 'WCsat', 'soil_depth', 'SHETRAN_ID'] 

df_soils_types = pivoted_order[lst_cols].copy().reset_index(drop=True)

# Create list of soil types ID
# This create the list without comparing. The comparison between soil properties will happen at latter stage
lst_soil_type = []
for i in range(0, len(df_soils_types)):
    # lst_soil_type.append(f'st_{str(i).zfill(5)}')
    lst_soil_type.append(i + 1)
    
# Sort by category and soil layer
# df_soils_types = df_soils_types.sort_values(by=['soil_depth'])

# Add list to dataframe
df_soils_types['soil_type_ID'] = lst_soil_type

# Add soil category column
df_soils_types['soil_category_ID'] = np.nan

# df_soils_types.sort_index()

df_soils_types

property_name,lat,lon,ALFA,Ksat,N,WCres,WCsat,soil_depth,SHETRAN_ID,soil_type_ID,soil_category_ID
0,848266,709018,0.01,5.82,1.36,0.04,0.61,0_5cm,-9999,1,NaN
1,848266,709018,0.02,5.82,1.36,0.04,0.60,5_15cm,-9999,2,NaN
2,848266,709018,0.02,6.15,1.38,0.04,0.59,15_30cm,-9999,3,NaN
3,848266,709018,0.01,2.09,1.43,0.04,0.59,30_60cm,-9999,4,NaN
4,848266,709018,0.01,2.04,1.42,0.04,0.57,60_100cm,-9999,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5581,888266,783018,0.02,8.48,1.38,0.04,0.57,15_30cm,-9999,5582,NaN
5582,888266,783018,0.02,3.38,1.44,0.04,0.57,30_60cm,-9999,5583,NaN
5583,888266,783018,0.02,3.62,1.44,0.04,0.56,60_100cm,-9999,5584,NaN
5584,888266,783018,0.02,3.82,1.44,0.04,0.55,100_200cm,-9999,5585,NaN


#### Check for duplicate type of soils

In [13]:
df_soils_types_final = df_soils_types.copy()

# List of soild characteristics names
lst_cols_charact =['ALFA', 'Ksat', 'N', 'WCres', 'WCsat']

# Create dataframe with soil characteristics only
df_check = df_soils_types[lst_cols_charact].copy().reset_index(drop=True)
df_check

property_name,ALFA,Ksat,N,WCres,WCsat
0,0.01,5.82,1.36,0.04,0.61
1,0.02,5.82,1.36,0.04,0.60
2,0.02,6.15,1.38,0.04,0.59
3,0.01,2.09,1.43,0.04,0.59
4,0.01,2.04,1.42,0.04,0.57
...,...,...,...,...,...
5581,0.02,8.48,1.38,0.04,0.57
5582,0.02,3.38,1.44,0.04,0.57
5583,0.02,3.62,1.44,0.04,0.56
5584,0.02,3.82,1.44,0.04,0.55


### 3.5. Create the soild type and soil categories
This creates a dataframe with all the characteristics for each grid

In [15]:
# List of soild type ID and category ID
# This will be used to avoid repetition in a further loop
lst_soil_type_ID = []
# lst_soil_ctgr_ID = []
cnt_cat = 1

# Creating the process of comparison through the whole dataframe
# for i in range(20):
for i in range(len(df_soils_types)):
    # empty variables
    tmp_lst = []
    tmo_df = []
    
    # Populate temp dataframe and remove row for dataframe
    tmp_df = pd.DataFrame()
    tmp_df = df_check.copy()
    
    #######################################################
    ### WORKING WITH THE SOIL TYPE ###
    #######################################################
    
    # Create soil type ID variable
    soil_type_ID = df_soils_types_final.loc[i, ['soil_type_ID']].values[0]
    
    # Get row as a list and remove from dataframe
    tmp_lst = tmp_df.values.tolist()[i]
    
    # Remove the row (tmp_lst) from dataframe before comparison
    tmp_df = tmp_df.drop(index=i)
    
    # Check if the set of soil characteristics (tmp_lst) appears again in the dataframe
    '''This takes the list of soil characteristics and compares it to each row of the dataframe individually. It returns
    a dataframe with the rows that are equal to the list'''
    duplicate_df = df_check.loc[df_check.isin(tmp_lst).astype(int).sum(axis=1) == len(tmp_lst), :]
    # duplicate_df = tmp_df.loc[tmp_df.isin(tmp_lst).astype(int).sum(axis=1) == len(tmp_lst), :]
    
    # check if duplicate df is not empty - returns true of empty
    flag = duplicate_df.empty
    
    # if the duplicate_df is not empty then make sure the rows have the same soil type
    if flag is False:
        
        # Check the soil type with the lst of soil types already checked
        if (soil_type_ID not in lst_soil_type_ID):
            '''For example if we are in row n+1 and the soild ID has already been changed for the one in 
            row n, then we don't want to redo that row again'''
            
            # Store soil type ID in list to avoid repetition in future loops
            lst_soil_type_ID.append(soil_type_ID)
            
            # Add the soil type ID to the rows with equal soil characteristics
            '''Here the code uses de index inside the soil duplicate dataframe to change the soil_type_ID
            in the final dataframe using the index and the soil_type_ID of the value that is currently being read'''
            for value in list(duplicate_df.index.values):
                df_soils_types_final.loc[value, 'soil_type_ID'] = soil_type_ID
    
    #######################################################
    ### WORKING WITH THE SOIL CATEGORY###
    ####################################################### 
    
    # Create soil type ID variable
    soil_category_ID = df_soils_types_final.loc[i, ['soil_category_ID']].values[0]
    
    # Create coordinates (soild category) as a list - ready for comparing
    lst_soil_categor = list(df_soils_types_final.loc[i, ['lat', 'lon']])
    
    # Find the grids where the set of coordinates are duplicate 
    '''This takes the list of soil characteristics and compares it to each row of the dataframe individually. It returns
    a dataframe with the rows that are equal to the list based on the lat and lon i.e the different soil depths'''
    
    grid_depths = df_soils_types_final.loc[df_soils_types_final.isin(lst_soil_categor).sum(axis=1) == len(lst_soil_categor), :]
    
    # check if duplicate df is not empty - returns true of empty
    flag_cat = grid_depths.empty
    
    # if the duplicate_df is not empty then make sure the rows have the same soil type
    
    if flag_cat is False:
        # Check the category is not nan
        if (pd.isna(soil_category_ID) is True):
            # print(grid_depths)
                        
            # Add the soil category ID to the rows with equal coordinates / grid
            '''Here the code uses de index inside the soil duplicate dataframe to change the soil_category_ID
            in the final dataframe using the index and the soil_category_ID of the value that is currently being read'''
            
            for value in list(grid_depths.index.values):
                # print(value)
                df_soils_types_final.loc[value, 'soil_category_ID'] = int(cnt_cat)
                
            # Increase the soil category ID
            cnt_cat += 1
    
    
    
# Make sure category ID is Integer
df_soils_types_final['soil_category_ID'] = df_soils_types_final['soil_category_ID'].astype(int)

# print(lst_soil_categor)
# print(pd.isna(soil_category_ID))
# grid_depths
df_soils_types_final.head(20)



property_name,lat,lon,ALFA,Ksat,N,WCres,WCsat,soil_depth,SHETRAN_ID,soil_type_ID,soil_category_ID
0,848266,709018,0.01,5.82,1.36,0.04,0.61,0_5cm,-9999,1,1
1,848266,709018,0.02,5.82,1.36,0.04,0.60,5_15cm,-9999,2,1
2,848266,709018,0.02,6.15,1.38,0.04,0.59,15_30cm,-9999,3,1
3,848266,709018,0.01,2.09,1.43,0.04,0.59,30_60cm,-9999,4,1
4,848266,709018,0.01,2.04,1.42,0.04,0.57,60_100cm,-9999,5,1
5,848266,709018,0.01,2.12,1.42,0.04,0.57,100_200cm,-9999,6,1
6,848266,709018,0.01,0.03,5.00,0.20,0.30,200_2000cm,-9999,7,1
7,848266,711018,0.01,5.80,1.35,0.04,0.60,0_5cm,-9999,8,2
8,848266,711018,0.01,5.41,1.35,0.04,0.59,5_15cm,-9999,9,2
9,848266,711018,0.02,5.78,1.37,0.04,0.59,15_30cm,-9999,10,2


## 4. Create XML
This is the library file

### 4.1. Soil Property and Soil Details

In [14]:
df_xml_v1 = df_soils_types_final.copy()

## Create the depth column
depth_values = [0.05, 0.15, 0.30, 0.60, 1.00, 2.00, 20.00]
layer_values = [1, 2, 3, 4, 5, 6, 7]
depth_conditions = [
    df_xml_v1['soil_depth'] == '0_5cm', 
    df_xml_v1['soil_depth'] == '5_15cm',
    df_xml_v1['soil_depth'] == '15_30cm',
    df_xml_v1['soil_depth'] == '30_60cm',
    df_xml_v1['soil_depth'] == '60_100cm',
    df_xml_v1['soil_depth'] == '100_200cm',
    df_xml_v1['soil_depth'] == '200_2000cm'
    ]

# Adding the soil depth column
df_xml_v1['soil_depth_meters'] = np.select(depth_conditions, depth_values)

# Adding the layer value column
df_xml_v1['soil_layer'] = np.select(depth_conditions, layer_values)

# Add soil number column based on the index
df_xml_v1['soil_number'] = df_xml_v1.index.values + 1


# Create soil properties column ready for xml
lst_soil_properties = ['soil_number', 'soil_type_ID',
                       'WCsat', 'WCres', 'Ksat',
                        'ALFA','N']

# Create soil properties column ready for xml
lst_soil_details = ['soil_category_ID', 'soil_layer',
                    'soil_type_ID', 'soil_depth_meters']

# Create Soil Property and Detail columns
df_xml_v1['SoilProperty'] = df_xml_v1[lst_soil_properties].values.tolist()
df_xml_v1['SoilDetail'] = df_xml_v1[lst_soil_details].values.tolist()

# Sort by category and soil layer
df_xml_v1 = df_xml_v1.sort_values(by=['soil_category_ID', 'soil_layer'])

display(df_xml_v1)

property_name,lat,lon,ALFA,Ksat,N,WCres,WCsat,soil_depth,SHETRAN_ID,soil_type_ID,soil_category_ID,soil_depth_meters,soil_layer,soil_number,SoilProperty,SoilDetail
0,848266,709018,0.013368,5.819515,1.355975,0.041,0.611368,0_5cm,-9999,st_00000,1.0,0.05,1,1,"[1, st_00000, 0.611367901234568, 0.041, 5.8195...","[1.0, 1, st_00000, 0.05]"
1,848266,709018,0.015981,5.821532,1.364080,0.041,0.600102,5_15cm,-9999,st_00001,1.0,0.15,2,2,"[2, st_00001, 0.6001024691358026, 0.041, 5.821...","[1.0, 2, st_00001, 0.15]"
2,848266,709018,0.020095,6.145344,1.378635,0.041,0.591844,15_30cm,-9999,st_00002,1.0,0.30,3,3,"[3, st_00002, 0.5918444444444445, 0.041, 6.145...","[1.0, 3, st_00002, 0.3]"
3,848266,709018,0.013593,2.092914,1.431116,0.041,0.586683,30_60cm,-9999,st_00003,1.0,0.60,4,4,"[4, st_00003, 0.5866827160493827, 0.041, 2.092...","[1.0, 4, st_00003, 0.6]"
4,848266,709018,0.013330,2.037880,1.418099,0.041,0.574720,60_100cm,-9999,st_00004,1.0,1.00,5,5,"[5, st_00004, 0.5747197530864198, 0.041, 2.037...","[1.0, 5, st_00004, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5581,888266,783018,0.020042,8.484741,1.379025,0.041,0.574916,15_30cm,-9999,st_05581,798.0,0.30,3,5582,"[5582, st_05581, 0.574916049382716, 0.041, 8.4...","[798.0, 3, st_05581, 0.3]"
5582,888266,783018,0.015065,3.377254,1.444694,0.041,0.565956,30_60cm,-9999,st_05582,798.0,0.60,4,5583,"[5583, st_05582, 0.5659555555555555, 0.041, 3....","[798.0, 4, st_05582, 0.6]"
5583,888266,783018,0.015942,3.617784,1.444964,0.041,0.559210,60_100cm,-9999,st_05583,798.0,1.00,5,5584,"[5584, st_05583, 0.5592098765432099, 0.041, 3....","[798.0, 5, st_05583, 1.0]"
5584,888266,783018,0.015917,3.824995,1.439658,0.041,0.552369,100_200cm,-9999,st_05584,798.0,2.00,6,5585,"[5585, st_05584, 0.5523691358024692, 0.041, 3....","[798.0, 6, st_05584, 2.0]"


### 4.2. Change Soil Property and Soil Detail to list to be added to the final dictionary

In [15]:
lst_soilDet_header = config.get('xml', 'soilDet_header').split(',')
lst_soilDet_header

['SoilCategory', 'SoilLayer', 'SoilType', 'DepthAtLayerBase']

In [17]:
'''Changing the column to list generates a list of lists. Before converting to XML
it is necessary to change each list to a single string element separated by a comma.
The final output should be a list where each element is the list of soil properties
as a string. It should go from a list of lists to a simple list of elements'''

lst_soilProp_header = config.get('xml', 'soilProp_header').split(',')
lst_soilDet_header = config.get('xml', 'soilDet_header').split(',')

# Soil Property to list of list
lst_soilPrprty_xml = list(df_xml_v1.SoilProperty)

# Changing to list of list ready for dicitonary
for i, lst_item in enumerate(lst_soilPrprty_xml):
    lst_soilPrprty_xml[i] = ", ".join([str(item) for item in lst_item])

# Soil Detail to list of list
lst_soilDetail_xml = list(df_xml_v1.SoilDetail)

# Changing to list of list ready for dicitonary
for i, lst_item in enumerate(lst_soilDetail_xml):
    lst_soilDetail_xml[i] = ", ".join([str(item) for item in lst_item])

# Add header to the list of soil properties
lst_soilPrprty_xml = [", ".join([str(item) for item in lst_soilProp_header])] + lst_soilPrprty_xml

# Add header to the list of soil details
lst_soilDetail_xml = [", ".join([str(item) for item in lst_soilDet_header])] + lst_soilDetail_xml


### 4.3. Create the dict ready for XML

In [18]:
# Dict flie tags
dict_tags = {
    'ProjectFile':'catch_project_file',
    'CatchmentName':'hybas_sa_lev06_v1c_santiagoCali_cauca',
    'DEMMeanFileName':'final_dem_min_SHETRAN.txt',
    'DEMminFileName':'final_dem_mean_SHETRAN.txt',
    'MaskFileName':'final_catchment_mask_SHETRAN.txt',
    'VegMap':'final_land_cover_SHETRAN.txt',
    'SoilMap':'final_subsurface_SHETRAN.txt',
    'LakeMap':'final_lake_map_SHETRAN.txt',
    'PrecipMap':'final_total_precipitation_map_SHETRAN.txt',
    'PeMap':'final_potential_evaporation_map_SHETRAN.txt'
}

# Vegetation details
'''This will need to be reviewed and coded rather than having it hard coded into the script'''
lst_vegdetails = [
    'VegetationTypeNumber, VegetationType,'\
    'CanopyStorageCapacity, LeafAreaIndex,'\
    'MaximumRootingDepth, AE/PEatFieldCapacity,'
    'StricklerCoefficient',
    '1, Arable, 1.5, 1, 0.8, 0.6, 2.5',
    '2, BareGround, 0, 0, 0.1, 0.4, 3.0',
    '3, Grass, 1.5, 1, 1.0, 0.6, 20.0',
    '4, Forest, 5, 1, 1.8, 1.0, 1.0',
    '5, FloodedVegetation, 0, 0, 0.1, 0.4, 3.0',
    '6, Urban, 0.3, 0.3, 0.5, 0.4, 5.0'
]

end_dict_tags = {
    'InitialConditions':'0',
    'PrecipitationTimeSeriesData':'final_total_precipitation_202201_202207.csv',
    'PrecipitationTimeStep':1,
    'EvaporationTimeSeriesData':'final_potential_evaporation_202201_202207.csv',
    'EvaporationTimeStep':1,
    'MaxTempTimeSeriesData':'final_2m_temperature_max_202201_202207.csv',
    'MinTempTimeSeriesData':'final_2m_temperature_min_202201_202207.csv',
    'StartDay':'01',
    'StartMonth':'01',
    'StartYear':'2020',
    'EndDay':'31',
    'EndMonth':'07',
    'EndYear':'2020',
    'RiverGridSquaresAccumulated':'2',
    'DropFromGridToChannelDepth':'2',
    'MinimumDropBetweenChannels':'0.5',
    'RegularTimestep':'1.0',
    'IncreasingTimestep':'0.05',
    'SimulatedDischargeTimestep':'24.0',
    'SnowmeltDegreeDayFactor':'0.0002'    
}



# Add vegetation detail tag to dict
dict_tags.update(VegetationDetails=lst_vegdetails)

# Add soilt property tag to dict
dict_tags.update(SoilProperties=lst_soilPrprty_xml)

# Add soilt detail tag to dict
dict_tags.update(SoilDetails=lst_soilDetail_xml)

# Add end tag to dict
dict_tags.update(end_dict_tags)

dict_tags

{'ProjectFile': 'catch_project_file',
 'CatchmentName': 'hybas_sa_lev06_v1c_santiagoCali_cauca',
 'DEMMeanFileName': 'final_dem_min_SHETRAN.txt',
 'DEMminFileName': 'final_dem_mean_SHETRAN.txt',
 'MaskFileName': 'final_catchment_mask_SHETRAN.txt',
 'VegMap': 'final_land_cover_SHETRAN.txt',
 'SoilMap': 'final_subsurface_SHETRAN.txt',
 'LakeMap': 'final_lake_map_SHETRAN.txt',
 'PrecipMap': 'final_total_precipitation_map_SHETRAN.txt',
 'PeMap': 'final_potential_evaporation_map_SHETRAN.txt',
 'VegetationDetails': ['VegetationTypeNumber, VegetationType,CanopyStorageCapacity, LeafAreaIndex,MaximumRootingDepth, AE/PEatFieldCapacity,StricklerCoefficient',
  '1, Arable, 1.5, 1, 0.8, 0.6, 2.5',
  '2, BareGround, 0, 0, 0.1, 0.4, 3.0',
  '3, Grass, 1.5, 1, 1.0, 0.6, 20.0',
  '4, Forest, 5, 1, 1.8, 1.0, 1.0',
  '5, FloodedVegetation, 0, 0, 0.1, 0.4, 3.0',
  '6, Urban, 0.3, 0.3, 0.5, 0.4, 5.0'],
 'SoilProperties': ['SoilNumber, SoilType, SaturatedWaterContent, ResidualWaterContent, SaturatedConducti

### 4.4 Convert dict to XML file

In [20]:
# Conver dict to XML
my_item_func = lambda x: 'SoilProperty' if x == 'SoilProperties' else ('SoilDetail' if x == 'SoilDetails' else 'VegetationDetail')
xml = dicttoxml(dict_tags, custom_root='ShetranInput', item_func = my_item_func, attr_type = False)

# Saving to a file
# make sure to remove any indent - needed for the executable
root = ET.fromstring(xml)
tree = ET.ElementTree(root)
ET.indent(tree, space="")

treestring = "<?xml version='1.0'?>" + ET.tostring(root, encoding='unicode', method='xml')
with open(Path(dir_abs / f"shetran_data/model_input/catch_project_file.xml"), "w") as f:
    f.write(treestring)
 
print(parseString(xml).toprettyxml())

<?xml version="1.0" ?>
<ShetranInput>
	<ProjectFile>catch_project_file</ProjectFile>
	<CatchmentName>hybas_sa_lev06_v1c_santiagoCali_cauca</CatchmentName>
	<DEMMeanFileName>final_dem_min_SHETRAN.txt</DEMMeanFileName>
	<DEMminFileName>final_dem_mean_SHETRAN.txt</DEMminFileName>
	<MaskFileName>final_catchment_mask_SHETRAN.txt</MaskFileName>
	<VegMap>final_land_cover_SHETRAN.txt</VegMap>
	<SoilMap>final_subsurface_SHETRAN.txt</SoilMap>
	<LakeMap>final_lake_map_SHETRAN.txt</LakeMap>
	<PrecipMap>final_total_precipitation_map_SHETRAN.txt</PrecipMap>
	<PeMap>final_potential_evaporation_map_SHETRAN.txt</PeMap>
	<VegetationDetails>
		<VegetationDetail>VegetationTypeNumber, VegetationType,CanopyStorageCapacity, LeafAreaIndex,MaximumRootingDepth, AE/PEatFieldCapacity,StricklerCoefficient</VegetationDetail>
		<VegetationDetail>1, Arable, 1.5, 1, 0.8, 0.6, 2.5</VegetationDetail>
		<VegetationDetail>2, BareGround, 0, 0, 0.1, 0.4, 3.0</VegetationDetail>
		<VegetationDetail>3, Grass, 1.5, 1, 1.0, 0.6,

## 5. Create subfarce map
This is base in the category ID

### 5.1. Setting dataframe for final file

In [21]:
# 1. Copy df soil types to avoid any issues
df_subsurface = df_soils_types_final.copy()

# 2. Change the soil category for SHETRAN_ID equal to -9999
'''This is needed as the subsurface should be the same as the mask'''
df_subsurface.loc[df_subsurface.SHETRAN_ID == ND, 'soil_category_ID'] = ND

# 3. Drop any duplicates
'''This is needed as each grid has 7 rows'''
df_subsurface = df_subsurface.drop_duplicates(subset=['lat', 'lon', 'soil_category_ID'])

# 4. Only work with the columns needed
df_subsurface = pd.DataFrame(df_subsurface[['soil_category_ID', 'lat', 'lon']].copy())


# Pivoting dataframe to replicate SHETRAN format
# Pivoting dataframe using lon as column and lat as row
df_subsurface_pivot = df_subsurface.pivot(index='lat', columns='lon', values='soil_category_ID')
df_subsurface_pivot = df_subsurface_pivot.sort_index(ascending=False).round(0).astype(int)

df_subsurface_pivot

lon,709018,711018,713018,715018,717018,719018,721018,723018,725018,727018,...,765018,767018,769018,771018,773018,775018,777018,779018,781018,783018
lat,,,,,,,,,,,,,,,,,,,,,
888266,-9999,-9999,-9999,-9999,-9999,-9999,767,768,769,770,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
886266,-9999,-9999,-9999,-9999,-9999,-9999,729,730,731,732,...,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
884266,-9999,-9999,-9999,-9999,-9999,690,691,692,693,694,...,713,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
882266,-9999,-9999,-9999,-9999,651,652,653,654,655,656,...,675,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999
880266,-9999,-9999,-9999,-9999,613,614,615,616,617,618,...,637,638,639,-9999,-9999,-9999,-9999,-9999,-9999,-9999
878266,-9999,-9999,-9999,-9999,575,576,577,578,579,580,...,599,600,601,-9999,603,604,-9999,-9999,-9999,-9999
876266,-9999,534,535,536,537,538,539,540,541,542,...,561,562,563,564,565,566,567,568,-9999,-9999
874266,495,496,497,498,499,500,501,502,503,504,...,523,524,525,526,527,528,529,530,531,532
872266,457,458,459,460,461,462,463,464,465,466,...,485,486,487,488,489,490,491,492,493,494


### 5.2. Saving the final outpt as csv file

In [22]:
# Creating the text file for min elevation
utils.shetran_csv_file(dir_abs, 'final_subsurface_SHETRAN', df_subsurface_pivot, 'd')
print('SHETRAN Subsurface file created')

SHETRAN Subsurface file created
